# Baseline Prediction 

https://towardsdatascience.com/multivariate-time-series-forecasting-using-random-forest-2372f3ecbad1

# <span style='background :khaki' > Install & Importa Libraries </span>

In [1]:
### ***Enviroment Preparation***
# Install Pandas
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install lightgbm

!pip install dtale

!pip install sktime
!pip install sklego

#!pip install skforecast

# Update pip -- WARNING Resolution
!python.exe -m pip install --upgrade pip

In [2]:
### ***Imports***
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import dtale

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from datetime import datetime
from datetime import timedelta

from statsmodels.tsa.stattools import adfuller, kpss

from sklego.preprocessing import RepeatingBasisFunction

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import time
# Current Location.. !cd
#C:\Users\ghoyo\Desktop\TFM\Nuevo\Proyect

# Paths
data = r"C:\Users\ghoyo\Desktop\TFM4\Project\GeneratedDfs\dayly_rests.json"

#### <span style="background:skyblue"> Load Data <span>

In [3]:
df = pd.read_json(data)
df.shape

(2061, 1)

#### <span style="background:skyblue"> Little Analysis <span>


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2061 entries, 2017-04-04 to 2022-11-24
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   n_rests  2061 non-null   int64
dtypes: int64(1)
memory usage: 32.2 KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
n_rests,2061.0,5.581756,2.828039,0.0,4.0,5.0,7.0,18.0


In [6]:
df.head(4)

,n_rests
2017-04-04,5
2017-04-05,7
2017-04-06,7
2017-04-07,7


# <span style='background :khaki' > Baseline Method </span>

#### <span style="background:violet"> Functions <span>

#### Baseline 1 = Predict Past Value

In [7]:
# persistence model
def model_persistence(x):
    return x

In [23]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.999999 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [9]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None, num_splits=None):
        n_samples = len(X)
        k_fold_size = n_samples // num_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(num_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int((i + 1) * n_samples / num_splits)
            yield indices[start: mid], indices[mid + margin: stop]

### <span style="background:lightgreen"> Dummy Regresors - lag_1 <span>

In [12]:
# Create Base Lags
base_df = df.copy()
base_df['n-1'] = base_df['n_rests'].shift(1)
base_df.dropna(inplace = True)
base_df.reset_index(drop=True, inplace=True)

In [13]:
base_df.head(2)

,n_rests,n-1
0,7,5.0
1,7,7.0


In [14]:
# X DATA... 
X = base_df['n-1']

# Y DATA SPLIT
y = base_df['n_rests']

# Other Variables
test_size = 1
predict = []

# Train, Predict and error.
for x in X[-test_size:]:
    yhat = model_persistence(x)
    predict.append(yhat)
    
mse = mean_squared_error(y[-test_size:], predict)
mae = mean_absolute_error(y[-test_size:], predict)
    

In [15]:
print('Prediction = ', yhat, ' -- Vs -- ', 'Original = ', list(y[-test_size:])[0])
print('MAE = ', mae)
print('MSE = ', mse)

Prediction =  3.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


#### <span style="background:skyblue"> Baseline CV </span>

#### <span style="background:skyblue"> TSCV </span>

In [16]:
# X DATA... 
X = base_df['n-1']

# Y DATA SPLIT
y = base_df['n_rests']

# Other Variables
test_size = 1
predict = []
b_mse = []
b_mae = []
#Split, Train, Predict and error.

tscv = TimeSeriesSplit(max_train_size=None, n_splits = 300, test_size=test_size)

for train_index, test_index in tscv.split(X):
    predict = []
    print('TRAIN:', train_index, 'TEST:', test_index) 
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for x in X_test:
        yhat = model_persistence(x)
        predict.append(yhat)
        print('X_test = ', x, '-- Predict = ', yhat)
        
    print('Prediction = ', yhat, ' -- Vs -- ', 'Original = ', list(y[-test_size:])[0])
    b_mse.append(mean_squared_error(y_test, predict))
    b_mae.append(mean_absolute_error(y_test, predict))
    print('MAE = ', b_mae[-1])
    print('MSE = ', b_mse[-1])
    print('\n')
    
    
print('Mae Mean = ', np.mean(b_mae))
print('Mse Mean = ', np.mean(b_mse))
print('\n')


TRAIN: [   0    1    2 ... 1757 1758 1759] TEST: [1760]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 1758 1759 1760] TEST: [1761]
X_test =  3.0 -- Predict =  3.0
Prediction =  3.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 1759 1760 1761] TEST: [1762]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original =  2
MAE =  0.0
MSE =  0.0


TRAIN: [   0    1    2 ... 1760 1761 1762] TEST: [1763]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 1761 1762 1763] TEST: [1764]
X_test =  3.0 -- Predict =  3.0
Prediction =  3.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [   0    1    2 ... 1762 1763 1764] TEST: [1765]
X_test =  1.0 -- Predict =  1.0
Prediction =  1.0  -- Vs --  Original =  2
MAE =  3.0
MSE =  9.0


TRAIN: [   0    1    2 ... 1763 1764 1765] TEST: [1766]
X_test =  4.0 

MAE =  0.0
MSE =  0.0


TRAIN: [   0    1    2 ... 1834 1835 1836] TEST: [1837]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  0.0
MSE =  0.0


TRAIN: [   0    1    2 ... 1835 1836 1837] TEST: [1838]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  3.0
MSE =  9.0


TRAIN: [   0    1    2 ... 1836 1837 1838] TEST: [1839]
X_test =  1.0 -- Predict =  1.0
Prediction =  1.0  -- Vs --  Original =  2
MAE =  0.0
MSE =  0.0


TRAIN: [   0    1    2 ... 1837 1838 1839] TEST: [1840]
X_test =  1.0 -- Predict =  1.0
Prediction =  1.0  -- Vs --  Original =  2
MAE =  4.0
MSE =  16.0


TRAIN: [   0    1    2 ... 1838 1839 1840] TEST: [1841]
X_test =  5.0 -- Predict =  5.0
Prediction =  5.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [   0    1    2 ... 1839 1840 1841] TEST: [1842]
X_test =  7.0 -- Predict =  7.0
Prediction =  7.0  -- Vs --  Original =  2
MAE =  3.0
MSE =  9.0


TRAIN: [   0    1    2 ... 1840 1841 1842] TE

X_test =  1.0 -- Predict =  1.0
Prediction =  1.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [   0    1    2 ... 1935 1936 1937] TEST: [1938]
X_test =  3.0 -- Predict =  3.0
Prediction =  3.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 1936 1937 1938] TEST: [1939]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [   0    1    2 ... 1937 1938 1939] TEST: [1940]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original =  2
MAE =  0.0
MSE =  0.0


TRAIN: [   0    1    2 ... 1938 1939 1940] TEST: [1941]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [   0    1    2 ... 1939 1940 1941] TEST: [1942]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [   0    1    2 ... 1940 1941 1942] TEST: [1943]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original 

Prediction =  5.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 2036 2037 2038] TEST: [2039]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  3.0
MSE =  9.0


TRAIN: [   0    1    2 ... 2037 2038 2039] TEST: [2040]
X_test =  1.0 -- Predict =  1.0
Prediction =  1.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 2038 2039 2040] TEST: [2041]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [   0    1    2 ... 2039 2040 2041] TEST: [2042]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 2040 2041 2042] TEST: [2043]
X_test =  5.0 -- Predict =  5.0
Prediction =  5.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [   0    1    2 ... 2041 2042 2043] TEST: [2044]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRA

#### <span style="background:skyblue"> BCV </span>

In [24]:
# X DATA... 
X = base_df['n-1']

# Y DATA SPLIT
y = base_df['n_rests']


# Other Variables
test_size = 1
predict = []
b_mse = []
b_mae = []
#Split, Train, Predict and error.

tscv = BlockingTimeSeriesSplit(n_splits = 300)

for train_index, test_index in tscv.split(X):
    predict = []
    print('TRAIN:', train_index, 'TEST:', test_index) 
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for x in X_test:
        yhat = model_persistence(x)
        predict.append(yhat)
        print('X_test = ', x, '-- Predict = ', yhat)
        
    print('Prediction = ', yhat, ' -- Vs -- ', 'Original = ', list(y[-test_size:])[0])
    b_mse.append(mean_squared_error(y_test, predict))
    b_mae.append(mean_absolute_error(y_test, predict))
    print('MAE = ', b_mae[-1])
    print('MSE = ', b_mse[-1])
    print('\n')
    
    
print('Mae Mean = ', np.mean(b_mae))
print('Mse Mean = ', np.mean(b_mse))
print('\n')


TRAIN: [0 1 2 3 4] TEST: [5]
X_test =  5.0 -- Predict =  5.0
Prediction =  5.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [ 6  7  8  9 10] TEST: [11]
X_test =  0.0 -- Predict =  0.0
Prediction =  0.0  -- Vs --  Original =  2
MAE =  0.0
MSE =  0.0


TRAIN: [12 13 14 15 16] TEST: [17]
X_test =  10.0 -- Predict =  10.0
Prediction =  10.0  -- Vs --  Original =  2
MAE =  5.0
MSE =  25.0


TRAIN: [18 19 20 21 22] TEST: [23]
X_test =  8.0 -- Predict =  8.0
Prediction =  8.0  -- Vs --  Original =  2
MAE =  3.0
MSE =  9.0


TRAIN: [24 25 26 27 28] TEST: [29]
X_test =  9.0 -- Predict =  9.0
Prediction =  9.0  -- Vs --  Original =  2
MAE =  5.0
MSE =  25.0


TRAIN: [30 31 32 33 34] TEST: [35]
X_test =  6.0 -- Predict =  6.0
Prediction =  6.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [36 37 38 39 40] TEST: [41]
X_test =  5.0 -- Predict =  5.0
Prediction =  5.0  -- Vs --  Original =  2
MAE =  6.0
MSE =  36.0


TRAIN: [42 43 44 45 46] TEST: [47]
X_test =  3.0 -- Predict = 

X_test =  11.0 -- Predict =  11.0
Prediction =  11.0  -- Vs --  Original =  2
MAE =  3.0
MSE =  9.0


TRAIN: [606 607 608 609 610] TEST: [611]
X_test =  6.0 -- Predict =  6.0
Prediction =  6.0  -- Vs --  Original =  2
MAE =  4.0
MSE =  16.0


TRAIN: [612 613 614 615 616] TEST: [617]
X_test =  11.0 -- Predict =  11.0
Prediction =  11.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [618 619 620 621 622] TEST: [623]
X_test =  7.0 -- Predict =  7.0
Prediction =  7.0  -- Vs --  Original =  2
MAE =  3.0
MSE =  9.0


TRAIN: [624 625 626 627 628] TEST: [629]
X_test =  5.0 -- Predict =  5.0
Prediction =  5.0  -- Vs --  Original =  2
MAE =  0.0
MSE =  0.0


TRAIN: [630 631 632 633 634] TEST: [635]
X_test =  1.0 -- Predict =  1.0
Prediction =  1.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [636 637 638 639 640] TEST: [641]
X_test =  2.0 -- Predict =  2.0
Prediction =  2.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [642 643 644 645 646] TEST: [647]
X_test =  6.0

X_test =  11.0 -- Predict =  11.0
Prediction =  11.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [1254 1255 1256 1257 1258] TEST: [1259]
X_test =  10.0 -- Predict =  10.0
Prediction =  10.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [1260 1261 1262 1263 1264] TEST: [1265]
X_test =  4.0 -- Predict =  4.0
Prediction =  4.0  -- Vs --  Original =  2
MAE =  2.0
MSE =  4.0


TRAIN: [1266 1267 1268 1269 1270] TEST: [1271]
X_test =  1.0 -- Predict =  1.0
Prediction =  1.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [1272 1273 1274 1275 1276] TEST: [1277]
X_test =  10.0 -- Predict =  10.0
Prediction =  10.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [1278 1279 1280 1281 1282] TEST: [1283]
X_test =  11.0 -- Predict =  11.0
Prediction =  11.0  -- Vs --  Original =  2
MAE =  0.0
MSE =  0.0


TRAIN: [1284 1285 1286 1287 1288] TEST: [1289]
X_test =  12.0 -- Predict =  12.0
Prediction =  12.0  -- Vs --  Original =  2
MAE =  1.0
MSE =  1.0


TRAIN: [12

# <span style="background:lightgreen"> Como se puede ver, los Errores de este método son bastante pequeños, por lo que es posible que no haga falta ni realizar predicciones con RandomForest  <span>

# <span style="background:lightgreen"> Generando Time Features  <span>